In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pyemu

In [5]:
def normal(mean,stdev,num_pts=50):
    xstart = mean - (4.0 * stdev)
    xend = mean + (4.0 * stdev)
    x = np.linspace(xstart,xend,num_pts)


In [9]:
jco_file = os.path.join("freyberg","freyberg.jcb")
sc = pyemu.Schur(jco=jco_file)
par_sum = sc.get_parameter_summary(include_map=True)
par_sum

,post_expt,prior_expt,prior_stdev,post_stdev,is_log
hkr00c00,2.437136,2.438752,0.250000,0.249996,True
hkr00c01,2.433425,2.438752,0.250000,0.249999,True
hkr00c02,2.428642,2.438752,0.250000,0.249992,True
hkr00c03,2.425291,2.438752,0.250000,0.249985,True
hkr00c04,2.426592,2.438752,0.250000,0.249948,True
hkr00c05,2.410287,2.438752,0.250000,0.249918,True
hkr00c06,2.414624,2.438752,0.250000,0.249777,True
hkr00c07,2.444397,2.438752,0.250000,0.249547,True
hkr00c08,2.492612,2.438752,0.250000,0.249091,True
hkr00c09,2.553294,2.438752,0.250000,0.248889,True


In [7]:
sc.get_forecast_summary(include_map=True)    


,prior_stdev,post_stdev,prior_expt,post_expt
or28c05_1,0.879440,0.816476,18.25471,2.756905
or28c05_0,0.633264,0.325699,21.18428,8.613202
sw_gw_1,333.988556,333.124994,-971.35140,14271.347941
sw_gw_0,282.195029,224.585214,-3809.07710,6413.166479
